In [1]:
import numpy as np
from numpy.random import seed 
seed(1) 
from tensorflow import set_random_seed 
set_random_seed(2)
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import StratifiedKFold

#Stop warnings
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

Using TensorFlow backend.
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


The neural network model I got was the best one I've found so from just playing around with the parameters. However, it is from far being an optimal model.

# Wide Model

In [2]:
def baseline_model2():
    """Wide neural network model."""
    model = Sequential()
    np.random.seed(1337) # for reproducibility
    model.add(Dense(input_dim=6, output_dim = 60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(10000, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(1000, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(250, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [32]:
def baseline_model2():
    """Wide neural network model."""
    model = Sequential()
    np.random.seed(1337) # for reproducibility
    model.add(Dense(input_dim=6, output_dim = 30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(30, kernel_initializer='normal', activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(30, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(30, kernel_initializer='normal', activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(30, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
    adam = optimizers.adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='mean_squared_error', optimizer=adam)
    return model

In [4]:
data = pd.read_csv('ATM_Training.csv')
y = data.pop('Withdraw')
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.33, random_state=42)
X_train +=0.001
X_test +=0.001
scaler = StandardScaler()
X_tran = scaler.fit(X_train.iloc[:,0:2])

In [5]:
dat = X_tran.transform(X_train.iloc[:,0:2])
dat2 = X_tran.transform(X_test.iloc[:,0:2])

In [6]:
X_train.pop('Shops')
X_train.pop('ATMs')
X_test.pop('Shops')
X_test.pop('ATMs')
X_train += 0.01
X_test += 0.01
data1train = []
data2train = []
data1test = []
data2test = []
for i in range(len(dat)):
    data1train.append(dat[i][0])
    data2train.append(dat[i][1])
for i in range(len(dat2)):
    data1test.append(dat2[i][0])
    data2test.append(dat2[i][1])
    
X_train['Shops'] = data1train
X_train['ATM'] = data2train
X_test['Shops'] = data1test
X_test['ATM'] = data2test

In [43]:
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers.noise import GaussianNoise
from keras.layers import Dropout

def baseline_model4():
    """Wide neural network model."""
    model = Sequential()
    np.random.seed(1337) # for reproducibility
    model.add(Dense(input_dim=6, output_dim = 250, kernel_initializer='normal'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(1000, kernel_initializer='normal'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(250, kernel_initializer='normal'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [44]:
model3 = baseline_model4()
model3.fit(X_train.as_matrix(), y_train, nb_epoch=25, batch_size=50)

Epoch 1/25
14740/14740 [==============================] - 5s - loss: 230.2685     
Epoch 2/25
14740/14740 [==============================] - 5s - loss: 1.5270     
Epoch 3/25
14740/14740 [==============================] - 5s - loss: 0.4776     
Epoch 4/25
14740/14740 [==============================] - 5s - loss: 0.3841     
Epoch 5/25
14740/14740 [==============================] - 5s - loss: 0.3378     
Epoch 6/25
14740/14740 [==============================] - 5s - loss: 0.3259     
Epoch 7/25
14740/14740 [==============================] - 5s - loss: 0.3525     
Epoch 8/25
14740/14740 [==============================] - 5s - loss: 0.3310     
Epoch 9/25
14740/14740 [==============================] - 5s - loss: 0.3608     
Epoch 10/25
14740/14740 [==============================] - 7s - loss: 0.3307     
Epoch 11/25
14740/14740 [==============================] - 5s - loss: 0.3857     
Epoch 12/25
14740/14740 [==============================] - 5s - loss: 0.3628     
Epoch 13/25
14740/14740

In [45]:
MSE_nn = model3.evaluate(X_test.as_matrix(), y_test)

7008/7260 [===========================>..] - ETA: 0s

In [46]:
print('Final MSE is {mse}'.format(mse = MSE_nn))

Final MSE is 0.3419023290974378


In [ ]:
#1. Dense (init = Gaussian) -> Gaussian Noise(0.05->0.1) -> 
#PRELU or Leaky-RELU(0.1->0.3) -> BatchNormalisation -> Gaussian Dropout (0.1->0.3) -> 2. Dense and so on.

In [ ]:
#= NAdam (lr = 0.005, momentum = 0.995)

In [ ]:
#NAdam optimiser (nesterov Adam)
#Batch Norm
#PRELU
#Gaussian Noise/Dropout

# Best Model

In [ ]:
def baseline_model2():
    """Wide neural network model."""
    model = Sequential()
    np.random.seed(1337) # for reproducibility
    model.add(Dense(input_dim=6, output_dim = 30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(30, kernel_initializer='normal', activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(30, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(30, kernel_initializer='normal', activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(30, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

    
  
model2 = baseline_model2()
model2.fit(X_train.as_matrix(), y_train, nb_epoch=25, batch_size=50)
MSE_nn = model2.evaluate(X_test.as_matrix(), y_test)
print('Final MSE is {mse}'.format(mse = MSE_nn))

0.2642245148660066